In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [38]:
torch.manual_seed(42)

In [39]:
transform = transforms.Compose([
    transforms.Resize(224),   
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


In [40]:
train_set=torchvision.datasets.CIFAR10(root='./cifar dataset', train=True,download=True,transform=transform)
test_set=torchvision.datasets.CIFAR10(root='./cifar dataset', train=False,download=True,transform=transform)

In [41]:
trainloader = DataLoader(
    train_set, batch_size=32, shuffle=True,pin_memory=True
)

testloader = DataLoader(
    test_set, batch_size=32, shuffle=False,pin_memory=True
)

In [42]:
model = models.resnet18(pretrained=True)


c:\Users\GOURAB\myenv1\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\GOURAB\myenv1\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [45]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
model=model.to(device)

for param in model.fc.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

In [46]:
epochs=10
learning_rate=0.001

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-5)

In [20]:
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for batch_images,batch_labels in trainloader:
    batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
    output=model(batch_images)
    loss=criterion(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  avg_loss = running_loss / len(trainloader)
  print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")



Epoch [1/10], Loss: 0.6719
Epoch [2/10], Loss: 0.3933
Epoch [3/10], Loss: 0.2869
Epoch [4/10], Loss: 0.2172
Epoch [5/10], Loss: 0.1576
Epoch [6/10], Loss: 0.1161
Epoch [7/10], Loss: 0.0970
Epoch [8/10], Loss: 0.0805
Epoch [9/10], Loss: 0.0727
Epoch [10/10], Loss: 0.0587


In [21]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_images, batch_labels in testloader:
        batch_images = batch_images.to(device)
        labels = batch_labels.to(device)

        outputs = model(batch_images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 90.18%


In [35]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_images, batch_labels in trainloader:
        batch_images = batch_images.to(device)
        labels = batch_labels.to(device)

        outputs = model(batch_images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Train Accuracy: {accuracy:.2f}%")


Train Accuracy: 6.70%
